In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive/fast_ds/thesis/experimentation/Segmentation/modification_testing
!ls

Mounted at /content/gdrive
/content/gdrive/My Drive/fast_ds/thesis/experimentation/Segmentation/modification_testing
ResUNetplusplus


In [2]:
%cd ./ResUNetplusplus
!ls 

/content/gdrive/My Drive/fast_ds/thesis/experimentation/Segmentation/modification_testing/ResUNetplusplus
'Copy of execution_script.ipynb'   process_image_sample.py
 data				   __pycache__
 data_generator.py		   README.md
 deployment_model.py		   result
 execution_script.ipynb		   results.png
 files				   resume_training.py
 frozen_models			   Resunet++.png
 GFLOPs.ipynb			   resunet_pp_modified.py
 infer.py			   resunet.py
 metrics.py			   run_miouTesting.py
 m_resunet_modified.py		   run_modified.py
 m_resunet.py			   run.py
 new_data			   run_sample.py
 process_image_deployment.py	   unet.py
 process_image.py


In [3]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
from unet import Unet
from resunet import ResUnet
from m_resunet_modified import ResUnetPlusPlus
import resunet_pp_modified

## Unet
arch = Unet(input_size=512)
Unet = arch.build_model()
print(Unet.summary())
print("************************Unet*********************")
print(len(Unet.layers))

## ResUnet
arch = ResUnet(input_size=512)
ResUnet = arch.build_model()
print(ResUnet.summary())
print("************************ResUnet*********************")
print(len(ResUnet.layers))

## ResUnet++
arch = ResUnetPlusPlus(input_size=512)
ResUnet_pp = arch.build_model()
print(ResUnet_pp.summary())
print("************************ResUnet++*********************")
print(len(ResUnet_pp.layers))

## ResUnet++_sepConv
arch = resunet_pp_modified.ResUnetPlusPlus(input_size=512)
ResUnet_pp = arch.build_model()
print(ResUnet_pp.summary())
print("************************ResUnet++_sepConv*********************")
print(len(ResUnet_pp.layers))
ResUnet_pp.save("files/ResUnet_pp.h5")

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 512, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 512, 512, 16) 64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 512, 512, 16) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [ ]:


import tensorflow as tf
import keras.backend as K
from keras.applications.mobilenet import MobileNet

run_meta = tf.compat.v1.RunMetadata()
with tf.compat.v1.Session(graph=tf.compat.v1.Graph()) as sess:
    tf.compat.v1.keras.backend.set_session(sess)
    # net = MobileNet(alpha=.75, input_tensor=tf.compat.v1.placeholder('float32', shape=(1,32,32,3)))
    net = ResUnet_pp

    opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()    
    flops = tf.compat.v1.profiler.profile(sess.graph, run_meta=run_meta, cmd='op', options=opts)

    opts = tf.compat.v1.profiler.ProfileOptionBuilder.trainable_variables_parameter()    
    params = tf.compat.v1.profiler.profile(sess.graph, run_meta=run_meta, cmd='op', options=opts)

    print("{:,} --- {:,}".format(flops.total_float_ops, params.total_parameters))

# net.summary()
# import tensorflow as tf
# from tensorflow.keras.models import Sequential 
# from tensorflow.keras.layers import Dense 
# from keras.applications.mobilenet import MobileNet

# print(tf.__version__)
# def get_flops(model):
#     run_meta = tf.compat.v1.RunMetadata()
#     opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()

#     # We use the Keras session graph in the call to the profiler.
#     flops = tf.compat.v1.profiler.profile(graph=tf.compat.v1.keras.backend.get_session().graph,
#                                 run_meta=run_meta, cmd='op', options=opts)

#     return flops.total_float_ops  # Prints the "flops" of the model.

# # .... Define your model here ....
# # model = Sequential() 
# # model.add(Dense(8, activation = 'softmax')) 
# model = MobileNet(alpha=.75, input_tensor=tf.compat.v1.placeholder('float32', shape=(1,32,32,3)))
# print(get_flops(model))



0 --- 0
Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_62 (Conv2D)              (None, 256, 256, 16) 448         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_62 (BatchNo (None, 256, 256, 16) 64          conv2d_62[0][0]                  
__________________________________________________________________________________________________
activation_52 (Activation)      (None, 256, 256, 16) 0           batch_normalization_62[0][0]     
_______________________________________________________________________________

In [4]:
## GFLOPs
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import CustomObjectScope
from data_generator import *
from metrics import dice_coef, dice_loss, miou_coef, miou_loss

def get_flops(model_h5_path):
    session = tf.compat.v1.Session()
    graph = tf.compat.v1.get_default_graph()


    with graph.as_default():
        with session.as_default():
          with CustomObjectScope({'dice_loss': dice_loss, 'dice_coef': dice_coef,'miou_loss':miou_loss,'miou_coef':miou_coef}):
            model = load_model(model_h5_path)
            # model = tf.keras.models.load_model(model_h5_path)

          run_meta = tf.compat.v1.RunMetadata()
          opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()

          # Optional: save printed results to file
          # flops_log_path = os.path.join(tempfile.gettempdir(), 'tf_flops_log.txt')
          # opts['output'] = 'file:outfile={}'.format(flops_log_path)

          # We use the Keras session graph in the call to the profiler.
          flops = tf.compat.v1.profiler.profile(graph=graph,
                                                run_meta=run_meta, cmd='op', options=opts)
          print(model.summary())
          return flops.total_float_ops

model_path = "files/unet_miou.h5"
get_flops(model_path)


# Total params: 3,047,265
# Trainable params: 3,039,329
# Non-trainable params: 7,936
# __________________________________________________________________________________________________
# None
# 6070057

# ResUnet_pp_sepConv 6070057
# daspp_relu_miou 10024898
# daspp_miou 10024898
# miou 8718068


# Total params: 4,371,265
# Trainable params: 4,363,329
# Non-trainable params: 7,936
# __________________________________________________________________________________________________
# None
# 8718068


# Total params: 5,024,705
# Trainable params: 5,017,313
# Non-trainable params: 7,392
# __________________________________________________________________________________________________
# None
# 10024898

# ____________________________________________________________________________________________________
# Unet
# Total params: 3,588,997
# Trainable params: 3,585,027
# Non-trainable params: 3,970
# __________________________________________________________________________________________________
# None
# 7165148

Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 16) 64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 256, 16) 0           batch_normalization[0][0]       

7165148

In [5]:
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import CustomObjectScope
from data_generator import *
from metrics import dice_coef, dice_loss, miou_coef, miou_loss

def mask_to_3d(mask):
    mask = np.squeeze(mask)
    mask = [mask, mask, mask]
    mask = np.transpose(mask, (1, 2, 0))
    return mask

if __name__ == "__main__":
    model_path = "files/unet_miou.h5"
    save_path = "result"
    test_path = "new_data/Kvasir-SEG/test/"

    image_size = 256
    batch_size = 1

    test_image_paths = glob(os.path.join(test_path, "images", "*"))
    test_mask_paths = glob(os.path.join(test_path, "masks", "*"))
    test_image_paths.sort()
    test_mask_paths.sort()

    ## Create result folder
    try:
        os.mkdir(save_path)
    except:
        pass

    ## Model
    with CustomObjectScope({'dice_loss': dice_loss, 'dice_coef': dice_coef,'miou_loss':miou_loss,'miou_coef':miou_coef}):
        model = load_model(model_path)

    ## Test
    print("Test Result: ")
    test_steps = len(test_image_paths)//batch_size
    test_gen = DataGen(image_size, test_image_paths, test_mask_paths, batch_size=batch_size)
    model.evaluate_generator(test_gen, steps=test_steps, verbose=1)


Test Result: 
Instructions for updating:
Please use Model.evaluate, which supports generators.
100/100 [==============================] - 38s 382ms/step - loss: 0.4047 - recall: 0.7523 - precision: 0.8452 - dice_coef: 0.7191 - mean_io_u: 0.4246 - miou_coef: 0.5953


In [ ]:
#coding=utf-8
'''
The complexity of the model test
 Model position can be replaced.
'''
from tensorflow.python.framework import graph_util
import tensorflow as tf
# from tensorflow.contrib.layers import flatten
from tensorflow.keras.layers import Flatten
model_h5_path = "files/daspp_relu_miou.h5"

# tf.compat.v1.disable_eager_execution()

def stats_graph(graph):
    # flops = tf.profiler.profile(graph, options=tf.profiler.ProfileOptionBuilder.float_operation())
    flops = tf.compat.v1.profiler.profile(graph, options=tf.compat.v1.profiler.ProfileOptionBuilder.float_operation())
    # params = tf.profiler.profile(graph, options=tf.profiler.ProfileOptionBuilder.trainable_variables_parameter())
    params = tf.compat.v1.profiler.profile(graph, options=tf.compat.v1.profiler.ProfileOptionBuilder.trainable_variables_parameter())
    print('GFLOPs: {};    Trainable params: {}'.format(flops.total_float_ops/1000000000.0, params.total_parameters))
 
def load_pb(pb):
    with tf.gfile.GFile(pb, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name='')
        return graph
 
 
with tf.Graph().as_default() as graph:
  with CustomObjectScope({'dice_loss': dice_loss, 'dice_coef': dice_coef,'miou_loss':miou_loss,'miou_coef':miou_coef}):
    model = load_model(model_h5_path)
         #Models start at # ××××××××××××××××××××××××××××
    # ***** (1) Create Graph *****
   
         # End model ××××××××××××××××××××××××××××
 
    print('stats before freezing')
    stats_graph(graph)
    with tf.compat.v1.Session() as sess:
        sess.run(tf.compat.v1.global_variables_initializer())
        # ***** (2) freeze graph *****
        output_graph = graph_util.convert_variables_to_constants(sess, graph.as_graph_def(), ['output'])
        with tf.gfile.GFile('graph.pb', "wb") as f:
            f.write(output_graph.SerializeToString())
# ***** (3) Load frozen graph *****
with tf.Graph().as_default() as graph:
    graph = load_pb('./graph.pb')
    print('stats after freezing')
    stats_graph(graph)

stats before freezing


168 ops no flops stats due to incomplete shapes.
168 ops no flops stats due to incomplete shapes.


GFLOPs: 0.010024898;    Trainable params: 5017313


AssertionError: ignored

In [ ]:
#https://medium.com/@sebastingarcaacosta/how-to-export-a-tensorflow-2-x-keras-model-to-a-frozen-and-optimized-graph-39740846d9eb
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
import numpy as np
#path of the directory where you want to save your model
frozen_out_path = 'files/'
# name of the .pb file
frozen_graph_filename = "frozen_daspp_relu_miou"
model_h5_path = "files/daspp_relu_miou.h5" # Your model path

with CustomObjectScope({'dice_loss': dice_loss, 'dice_coef': dice_coef,'miou_loss':miou_loss,'miou_coef':miou_coef}):
    model = load_model(model_h5_path)

# Convert Keras model to ConcreteFunction
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))
# Get frozen ConcreteFunction
frozen_func = convert_variables_to_constants_v2(full_model)
frozen_func.graph.as_graph_def()
layers = [op.name for op in frozen_func.graph.get_operations()]
print("-" * 60)
print("Frozen model layers: ")
for layer in layers:
    print(layer)
print("-" * 60)
print("Frozen model inputs: ")
print(frozen_func.inputs)
print("Frozen model outputs: ")
print(frozen_func.outputs)
# Save frozen graph to disk
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir=frozen_out_path,
                  name=f"{frozen_graph_filename}.pb",
                  as_text=False)
# Save its text representation
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir=frozen_out_path,
                  name=f"{frozen_graph_filename}.pbtxt",
                  as_text=True)

Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
to_proto not supported in EAGER mode.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
to_proto not supported in EAGER mode.
------------------------------------------------------------
Frozen model layers: 
x
functional_1/conv2d/Conv2D/ReadVariableOp/resource
functional_1/conv2d/Conv2D/ReadVariableOp
functional_1/conv2d/Conv2D
functional_1/conv2d/BiasAdd/ReadVariableOp/resource
functional_1/conv2d/BiasAdd/ReadVariableOp
functional_1/conv2d/BiasAdd
functional_1/batch_normalization/ReadVariableOp/resource
functional_1/batch_normalization/ReadVariableOp
functional_1/batch_normalization/ReadVariableOp_1/resource
functional_1/batch_normalization/ReadVariableOp_1
functional_1/batch_normalization/FusedBatchNormV3/ReadVariableOp/resource
functional_1/batch_normaliza

'files/frozen_daspp_relu_miou.pbtxt'

In [ ]:
def load_pb(pb):
    with tf.compat.v2.io.gfile.GFile(pb, "rb") as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name='')
        return graph

with tf.Graph().as_default() as graph:
    graph = load_pb('./files/frozen_daspp_relu_miou.pb')
    print('stats after freezing')
    stats_graph(graph)

stats after freezing


142 ops no flops stats due to incomplete shapes.
142 ops no flops stats due to incomplete shapes.


GFLOPs: 1e-08;    Trainable params: 0


In [ ]:
def load_pb(pb):
    with tf.compat.v2.io.gfile.GFile(pb, "rb") as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name='')
        return graph

with tf.Graph().as_default() as graph:
    graph = load_pb('./files/frozen_daspp_relu_miou.pb')
    print('stats after freezing')
    stats_graph(graph)

DecodeError: ignored

In [ ]:
import tensorflow as tf
from tensorflow.python.framework import graph_util
model_h5_path = "files/daspp_relu_miou.h5"

def load_pb(pb):
    with tf.gfile.GFile(pb, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name='')
        return graph

# ***** (1) Create Graph *****
g = tf.compat.v1.get_default_graph()
sess = tf.compat.v1.Session(graph=g)
with g.as_default():
    # A = tf.Variable(initial_value=tf.random_normal([25, 16]))
    # B = tf.Variable(initial_value=tf.random_normal([16, 9]))
    # C = tf.matmul(A, B, name='output')
    with CustomObjectScope({'dice_loss': dice_loss, 'dice_coef': dice_coef,'miou_loss':miou_loss,'miou_coef':miou_coef}):
      model = load_model(model_h5_path)
    sess.run(tf.compat.v1.global_variables_initializer())

    run_meta = tf.compat.v1.RunMetadata()
    opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
    # flops = tf.compat.v1.profiler.profile(graph = g, options = opts , run_meta=run_meta,)
    # print('FLOP before freezing', flops.total_float_ops)
# *****************************        

# ***** (2) freeze graph *****
output_graph_def = tf.compat.v1.graph_util.convert_variables_to_constants(sess, g.as_graph_def(), ['functional_1/activation_28/Sigmoid'])

with tf.gfile.GFile('graph.pb', "wb") as f:
    f.write(output_graph_def.SerializeToString())
# *****************************


# ***** (3) Load frozen graph *****
g2 = load_pb('./graph.pb')
with g2.as_default():
    flops = tf.profiler.profile(g2, options = tf.profiler.ProfileOptionBuilder.float_operation())
    print('FLOP after freezing', flops.total_float_ops)

AssertionError: ignored

In [ ]:
def get_flops(model_h5_path):
    session = tf.compat.v1.Session()
    graph = tf.compat.v1.get_default_graph()


    with graph.as_default():
        with session.as_default():
          with CustomObjectScope({'dice_loss': dice_loss, 'dice_coef': dice_coef,'miou_loss':miou_loss,'miou_coef':miou_coef}):
            model = load_model(model_h5_path)
            # model = tf.keras.models.load_model(model_h5_path)

          run_meta = tf.compat.v1.RunMetadata()
          opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()

          # Optional: save printed results to file
          # flops_log_path = os.path.join(tempfile.gettempdir(), 'tf_flops_log.txt')
          # opts['output'] = 'file:outfile={}'.format(flops_log_path)

          # We use the Keras session graph in the call to the profiler.
          flops = tf.compat.v1.profiler.profile(graph=graph,
                                                run_meta=run_meta, cmd='op', options=opts)
          print(model.summary())
          return flops.total_float_ops

In [ ]:
#https://stackoverflow.com/questions/60974077/how-to-save-keras-model-as-frozen-graph
import tensorflow as tf
import os
from tensorflow.python.tools import freeze_graph
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(64, input_shape=(1,)))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='softmax'))
model.compile(optimizer='adam', loss='mse')
model.summary()

# Convert Keras model to ConcreteFunction
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype, name="yourInputName"))
# Get frozen ConcreteFunction
frozen_func = convert_variables_to_constants_v2(full_model)
frozen_func.graph.as_graph_def()
layers = [op.name for op in frozen_func.graph.get_operations()]
print("-" * 50)
print("Frozen model layers: ")
for layer in layers:
    print(layer)
print("-" * 50)
print("Frozen model inputs: ")
print(frozen_func.inputs)
print("Frozen model outputs: ")
print(frozen_func.outputs)
# Save frozen graph from frozen ConcreteFunction to hard drive
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir="./frozen_models",
                  name="frozen_graph.pb",
                  as_text=False)

### USAGE ##
def wrap_frozen_graph(graph_def, inputs, outputs, print_graph=False):
    def _imports_graph_def():
        tf.compat.v1.import_graph_def(graph_def, name="")

    wrapped_import = tf.compat.v1.wrap_function(_imports_graph_def, [])
    import_graph = wrapped_import.graph

    print("-" * 50)
    print("Frozen model layers: ")
    layers = [op.name for op in import_graph.get_operations()]
    if print_graph == True:
        for layer in layers:
            print(layer)
    print("-" * 50)

    return wrapped_import.prune(
        tf.nest.map_structure(import_graph.as_graph_element, inputs),
        tf.nest.map_structure(import_graph.as_graph_element, outputs))

## Example Usage ###
# Load frozen graph using TensorFlow 1.x functions
with tf.io.gfile.GFile("./frozen_models/frozen_graph.pb", "rb") as f:
    graph_def = tf.compat.v1.GraphDef()
    loaded = graph_def.ParseFromString(f.read())

# Wrap frozen graph to ConcreteFunctions
frozen_func = wrap_frozen_graph(graph_def=graph_def,
                                inputs=["yourInputName:0"],
                                outputs=["Identity:0"],
                                print_graph=True)
print("-" * 50)
print("Frozen model inputs: ")
print(frozen_func.inputs)
print("Frozen model outputs: ")
print(frozen_func.outputs)
# Get predictions for test images
predictions = frozen_func(yourInputName=tf.constant([[3.]]))
# Print the prediction for the first image
print("-" * 50)
print("Example prediction reference:")
print(predictions[0].numpy())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                128       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 2,753
Trainable params: 2,753
Non-trainable params: 0
_________________________________________________________________
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
to_proto not supported in EAGER mode.
Type is unsupported, or the types of the items don't match field type in C

In [ ]:
import tensorflow as tf
import os
from tensorflow.python.tools import freeze_graph
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

# model = tf.keras.Sequential()
# model.add(tf.keras.layers.Dense(64, input_shape=(1,)))
# model.add(tf.keras.layers.Dense(32, activation='relu'))
# model.add(tf.keras.layers.Dense(16, activation='relu'))
# model.add(tf.keras.layers.Dense(1, activation='softmax'))
# model.compile(optimizer='adam', loss='mse')
# model.summary()

with CustomObjectScope({'dice_loss': dice_loss, 'dice_coef': dice_coef,'miou_loss':miou_loss,'miou_coef':miou_coef}):
  model = load_model(model_h5_path)

# Convert Keras model to ConcreteFunction
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype, name="yourInputName"))
# Get frozen ConcreteFunction
frozen_func = convert_variables_to_constants_v2(full_model)
frozen_func.graph.as_graph_def()
layers = [op.name for op in frozen_func.graph.get_operations()]
print("-" * 50)
print("Frozen model layers: ")
for layer in layers:
    print(layer)
print("-" * 50)
print("Frozen model inputs: ")
print(frozen_func.inputs)
print("Frozen model outputs: ")
print(frozen_func.outputs)
# Save frozen graph from frozen ConcreteFunction to hard drive
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir="./frozen_models",
                  name="frozen_graph.pb",
                  as_text=False)

### USAGE ##
def wrap_frozen_graph(graph_def, inputs, outputs, print_graph=False):
    def _imports_graph_def():
        tf.compat.v1.import_graph_def(graph_def, name="")

    wrapped_import = tf.compat.v1.wrap_function(_imports_graph_def, [])
    import_graph = wrapped_import.graph

    print("-" * 50)
    print("Frozen model layers: ")
    layers = [op.name for op in import_graph.get_operations()]
    if print_graph == True:
        for layer in layers:
            print(layer)
    print("-" * 50)

    return wrapped_import.prune(
        tf.nest.map_structure(import_graph.as_graph_element, inputs),
        tf.nest.map_structure(import_graph.as_graph_element, outputs))

## Example Usage ###
# Load frozen graph using TensorFlow 1.x functions
with tf.io.gfile.GFile("./frozen_models/frozen_graph.pb", "rb") as f:
    graph_def = tf.compat.v1.GraphDef()
    loaded = graph_def.ParseFromString(f.read())

# Wrap frozen graph to ConcreteFunctions
frozen_func = wrap_frozen_graph(graph_def=graph_def,
                                inputs=["yourInputName:0"],
                                outputs=["Identity:0"],
                                print_graph=True)
print("-" * 50)
print("Frozen model inputs: ")
print(frozen_func.inputs)
print("Frozen model outputs: ")
print(frozen_func.outputs)
# Get predictions for test images
# predictions = frozen_func(yourInputName=tf.constant([[3.]]))
# Print the prediction for the first image
print("-" * 50)
# print("Example prediction reference:")
# print(predictions[0].numpy())

Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
to_proto not supported in EAGER mode.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
to_proto not supported in EAGER mode.
--------------------------------------------------
Frozen model layers: 
yourInputName
functional_1/conv2d/Conv2D/ReadVariableOp/resource
functional_1/conv2d/Conv2D/ReadVariableOp
functional_1/conv2d/Conv2D
functional_1/conv2d/BiasAdd/ReadVariableOp/resource
functional_1/conv2d/BiasAdd/ReadVariableOp
functional_1/conv2d/BiasAdd
functional_1/batch_normalization/ReadVariableOp/resource
functional_1/batch_normalization/ReadVariableOp
functional_1/batch_normalization/ReadVariableOp_1/resource
functional_1/batch_normalization/ReadVariableOp_1
functional_1/batch_normalization/FusedBatchNormV3/ReadVariableOp/resource
functional_1/batch_normali

In [ ]:
def load_pb(pb):
    with tf.compat.v2.io.gfile.GFile(pb, "rb") as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name='')
        return graph

with tf.Graph().as_default() as graph:
    graph = load_pb("./frozen_models/frozen_graph.pb")
    print('stats after freezing')
    stats_graph(graph)

stats after freezing


142 ops no flops stats due to incomplete shapes.
142 ops no flops stats due to incomplete shapes.


GFLOPs: 1e-08;    Trainable params: 0
